In [79]:
import torch
import os
from transformers import T5Tokenizer, T5ForConditionalGeneration, AutoConfig
from collections import Counter
import numpy as np
import bert_score

# 🔹 Set Model Directory
MODEL_DIR = r"C:\Users\LENOVO\Desktop\Projects\SafeMind AI\t5_finetuned_model"
device = "cuda" if torch.cuda.is_available() else "cpu"

# 🔹 Get Checkpoints
CHECKPOINT_PATHS = [os.path.join(MODEL_DIR, f"checkpoint-{i}") for i in range(1, 4)]
print(f"🔹 Found Checkpoints: {CHECKPOINT_PATHS}")

# 🔹 Load Tokenizer
tokenizer = T5Tokenizer.from_pretrained(CHECKPOINT_PATHS[-1])

# 🔹 Load Models
models = []
for i, path in enumerate(CHECKPOINT_PATHS):
    config = AutoConfig.from_pretrained(path)
    model = T5ForConditionalGeneration.from_pretrained(
        path, config=config, torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32
    ).to(device).eval()
    models.append(model)

print(f"✅ Loaded {len(models)} models for ensembling.")

# 🔹 Define Mental Health Keywords for Reinforcement
MENTAL_HEALTH_KEYWORDS = {
    "self-care", "therapy", "support", "healing", "emotions", 
    "mindfulness", "relaxation", "breathing", "coping", "balance"
}

# 🔹 Function for Enhanced Ensemble Prediction with Keyword Reinforcement
def ensemble_generate(prompt, num_return_sequences=3):
    input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(device)

    responses_with_scores = []
    
    for i, model in enumerate(models):
        temperature = max(0.3, 1 - (0.1 * len(models)))  # Adaptive temperature
        outputs = model.generate(
            input_ids,
            return_dict_in_generate=True,
            output_scores=True,
            max_length=300,
            num_beams=8,
            early_stopping=True,
            repetition_penalty=3.0,
            length_penalty=2.0,
            temperature=temperature,
            top_k=50,
            top_p=0.95,
            do_sample=True,
            num_return_sequences=num_return_sequences
        )

        # Assign confidence scores using logits if available
        if outputs.scores:
            logits = torch.stack(outputs.scores).mean().item()
        else:
            logits = 0  # Default if no score is provided

        model_weight = (i + 1) * 1.5

        for output in outputs.sequences:
            response = tokenizer.decode(output, skip_special_tokens=True).strip()
            confidence_score = logits * model_weight
            
            # 🔹 Apply Keyword Boosting
            keyword_count = sum(1 for word in MENTAL_HEALTH_KEYWORDS if word in response.lower())
            confidence_score += keyword_count * 2  # Boost responses with relevant keywords
            
            responses_with_scores.append((response, confidence_score))

    # 🔹 Weighted Voting for Best Response
    response_scores = Counter()
    for response, score in responses_with_scores:
        response_scores[response] += score

    # 🔹 Get Top Responses
    top_responses = [resp for resp, _ in response_scores.most_common(5)]

    if not top_responses:
        return "I'm sorry, but I couldn't generate a response."

    # 🔹 Apply BERTScore for Similarity Filtering
    P, R, F1 = bert_score.score(
        top_responses, [prompt] * len(top_responses), lang="en", model_type="bert-base-uncased"
    )

    best_response = top_responses[np.argmax(F1.cpu().numpy())]

    return best_response  # ✅ Correctly indented return statement


🔹 Found Checkpoints: ['C:\\Users\\LENOVO\\Desktop\\Projects\\SafeMind AI\\t5_finetuned_model\\checkpoint-1', 'C:\\Users\\LENOVO\\Desktop\\Projects\\SafeMind AI\\t5_finetuned_model\\checkpoint-2', 'C:\\Users\\LENOVO\\Desktop\\Projects\\SafeMind AI\\t5_finetuned_model\\checkpoint-3']
✅ Loaded 3 models for ensembling.


In [80]:
# 🔹 Example Usage
example_input = "Why do I feel so anxious all the time?"
print("🔹 AI Response:", ensemble_generate(example_input))


🔹 AI Response: why do you feel anxious all the time? i'm so sorry to hear that. i am not sure what is going on in your life.


In [81]:
# 🔹 Example Usage
example_input = "How can I calm myself down during a panic attack?"
print("🔹 AI Response:", ensemble_generate(example_input))


🔹 AI Response: panic attacks can be a great way to calm yourself down. the best way to calm yourself is to focus on what you are feeling and how you feel when you have panic attacks.


In [82]:
# 🔹 Example Usage
example_input = "What are some quick ways to reduce stress?"
print("🔹 AI Response:", ensemble_generate(example_input))


🔹 AI Response: there are some quick ways to reduce stress. the best way to reduce stress is to focus on focusing on what you're looking for in order to improve your mood and feel better.


In [83]:
# 🔹 Example Usage
example_input = "Why do I get nervous in social situations?"
print("🔹 AI Response:", ensemble_generate(example_input))


🔹 AI Response: i don't know why you feel nervous in social situations. i am not sure what to do, but i haven't heard of it before. i can't believe that there is something wrong with the situation.


In [84]:
# 🔹 Example Usage
example_input = "How do I deal with workplace stress?"
print("🔹 AI Response:", ensemble_generate(example_input))


🔹 AI Response: i'm not sure how to deal with stress in the workplace. there are many different types of things that you can do to address your stress. it is important to have a clear understanding of what is going on in your workplace and what is going on in your life. this will help you cope with some of the most stressful aspects of your workdays.


In [85]:
# 🔹 Example Usage
example_input = "Is it normal to feel anxious even when nothing is wrong?"
print("🔹 AI Response:", ensemble_generate(example_input))


🔹 AI Response: yes, it is normal to feel anxious even when nothing is wrong. i'm not sure whether or not there is anything wrong with your anxiety. you may want to know more about yourself and the reasons for this anxiety.


In [86]:
# 🔹 Example Usage
example_input = "How can I relax my mind before sleeping?"
print("🔹 AI Response:", ensemble_generate(example_input))


🔹 AI Response: the first step is to relax your mind before sleeping. it's important to have a clearer understanding of what you are feeling and how you feel about yourself. there are many ways to relax your mind before sleeping. this can be a great way to relax your mind before sleeping.


In [87]:
# 🔹 Example Usage
example_input = "What are some mindfulness techniques for anxiety relief?"
print("🔹 AI Response:", ensemble_generate(example_input))


🔹 AI Response: mindfulness techniques for anxiety relief are some of the best ways to reduce anxiety. i'm not sure what is going on in your life, but i don't know how to deal with it. meditation can help you cope with anxiety and improve your mood.
